In [ ]:
# 对文本预料进行处理，生成4个向量，其中需要将文本转化为词，在此基础上转化为主题模型，取出主题对应的词；文档对应的主题

In [1]:
import linecache
import pandas as pd
import os
from string import punctuation
import nltk
import re
from nltk.corpus import stopwords
import itertools
from collections import Counter
import numpy as np
with open('test_patent.txt', 'r') as f:
    patent_content=[]
    patent_author=[]
    patent_year=[]
    patent_institution=[]
    patent_IPC=[]    
    content_str=""
        
    title_str=""
    abstract_str=""
    
    line=f.readline()
    
    while not line.startswith('EF'):
        
        if line.startswith("TI "):
            title_str=line.replace("TI ","").strip().strip('\n')+" "
            #print(title_str)
            line=f.readline()
            
        elif line.startswith("AU "):
            #print line
            author_list=[line.replace("AU ","").strip().strip('\n')]
            line=f.readline()
            while line.startswith("   "):
                author_list.append(line.strip().strip('\n'))
                line=f.readline()                
            patent_author.append(author_list)
            print(patent_author)
        
        elif line.startswith("AE "):
            #print line
            institution_list=[line.replace("AE ","").strip().strip('\n')]
            line=f.readline()
            while line.startswith("   "):
                institution_list.append(line.strip().strip('\n'))
                line=f.readline()                
            patent_institution.append(institution_list)
            print(patent_institution)
        
        elif line.startswith("AB "):
            abstract_str=line.replace("AB    NOVELTY - ","").strip().strip('\n')+" "
            line=f.readline()
            while line.startswith("   "):                
                if line.startswith("   DETAILED DESCRIPTION - "):
                    line=f.readline()
                abstract_str=abstract_str+line.strip().strip('\n')+" "               
                #print(abstract_str)
                line=f.readline()
            if "DESCRIPTION OF DRAWING(S) - " in abstract_str:
                index=abstract_str.index("DESCRIPTION OF DRAWING(S) - ")
                abstract_str=abstract_str[:index]                
            abstract_str=abstract_str.replace("USE - ","").replace("ADVANTAGE - ","")
            #print(abstract_str)
        
        elif line.startswith("IP "):
            #print line
            patent_IPC_list=line.replace("IP ","").split(';')
            patent_IPC_short_list=(re.split(r'-', ipc.strip(), maxsplit=1)[0] for ipc in patent_IPC_list)
            patent_IPC_short_list=list(set(patent_IPC_short_list))
            patent_IPC.append(patent_IPC_short_list)
            print(patent_IPC)
            line=f.readline()
            
        elif line.startswith("AD "):
            #print line
            year=line[-5:].strip('\n')        
            patent_year.append(year)
            print(patent_year)
            line=f.readline()
            
        elif line.startswith("ER"):
            content_str=(title_str+abstract_str).strip('\n').strip()
            patent_content.append(content_str)
            print(patent_content)
            title_str=""
            abstract_str=""
            line=f.readline()
            
        else:    
            line=f.readline()
print('finish')

[['LUO F', 'DAN J']]
[['BEIJING SENSETIME TECHNOLOGY DEV CO LTD (BEIJ-Non-standard)', 'UNIV FOSHAN (UYFS-C)']]
[['G06T', 'H04N']]
['2016']
['Method for training convolutional neural network, involves adjusting network parameters of convolutional neural network according to first difference and second difference. The method involves detecting (S100) a raw sample image containing target object label information and a scrambled sample image corresponding to the original sample image based on a convolutional neural network to obtain the original image for the original image. A first prediction information of the target object in the sample image and second prediction information for the target object in the scrambled sample image. A first difference between the first prediction information and the annotation information, and a second difference between the first prediction information and the second prediction information are determined (S200). The network parameters of the convolutional n

In [2]:
class ProcessDwpiText(object):
    @classmethod
    def get_term(cls, sentence, grammar, pos):
        toks = cls.sent_tokenize(sentence)
        for tok in toks:
            for leaf in cls.leaves(tok, grammar, pos):
                term = [cls.normalise(w) for w, t in leaf if cls.acceptable_word(w)]
                print(term)
                yield term

    @staticmethod
    def leaves(toks, grammar, pos):
        """
        :param toks: 分句后的列表
        :param grammar:
        :param pos: 词性
        :return:
        """
        chunker = nltk.RegexpParser(grammar)
        postoks = nltk.tag.pos_tag(toks)
        tree = chunker.parse(postoks)
        # tree.draw()
        for subtree in tree.subtrees(lambda t: t.label() == pos):
            yield subtree.leaves()

    @staticmethod
    def sent_tokenize(sentence):
        # return:  [[单词，单词],[单词，单词],...] 每个字列表为句子
        for sentence in nltk.sent_tokenize(sentence):
            tokenize = nltk.word_tokenize(sentence)
            yield tokenize

    # 过滤单词对的规则
    @staticmethod
    def acceptable_word(word):
        return bool(not re.match(r'[{}]'.format(punctuation), word) and not re.search(r'\d',
                                                                                      word) and word.lower() not in stopwords.words(
            'english') and 2 <= len(word) <= 40)

    # 提取词干
    @staticmethod
    def normalise(word):
        lemmatizer = nltk.WordNetLemmatizer()
        word = lemmatizer.lemmatize(word)
        return word.lower()

In [3]:
patent_content_tokenized=[]
grammar = r"""
                        NBAR:
                            {<NN.*|JJ>*<NN.*>}  # Nouns and Adjectives, terminated with Nouns

                        NP:
                            {<NBAR>}
                            {<NBAR><IN><NBAR>}  # Above, connected with in/of/etc...;IN代表介词
                    """

for text in patent_content:
    content_term =(ProcessDwpiText.get_term(text, grammar, "NP"))
    term_list = itertools.chain((" ".join(ab) for ab in content_term))
    patent_content_tokenized.append(list(filter(None, term_list)))
print("patent_content_tokenized",patent_content_tokenized)

['method']
['convolutional', 'neural', 'network']
['network', 'parameter']
['convolutional', 'neural', 'network']
['first', 'difference']
['second', 'difference']
['method']
[]
['raw', 'sample', 'image']
['target', 'object', 'label', 'information']
['scrambled', 'sample', 'image']
['original', 'sample', 'image']
['convolutional', 'neural', 'network']
['original', 'image']
['original', 'image']
['first', 'prediction', 'information']
['target', 'object']
['sample', 'image']
['second', 'prediction', 'information']
['target', 'object']
['scrambled', 'sample', 'image']
['first', 'difference']
['first', 'prediction', 'information']
['annotation', 'information']
['second', 'difference']
['first', 'prediction', 'information']
['second', 'prediction', 'information']
[]
['network', 'parameter']
['convolutional', 'neural', 'network']
[]
['first', 'difference']
['second', 'difference']
['method']
['convolutional', 'neural', 'network']
['method']
['inter-frame', 'jitter']
['video', 'frame', 'image'

In [8]:
from collections import defaultdict
from pprint import pprint

frequency = defaultdict(int)
for text in patent_content_tokenized:
    for token in text:
        frequency[token] += 1
patent_content_tokenized = [[token for token in text if frequency[token] > 1] for text in patent_content_tokenized]
pprint(patent_content_tokenized)

[['method',
  'convolutional neural network',
  'network parameter',
  'convolutional neural network',
  'first difference',
  'second difference',
  'method',
  'scrambled sample image',
  'convolutional neural network',
  'original image',
  'original image',
  'first prediction information',
  'target object',
  'second prediction information',
  'target object',
  'scrambled sample image',
  'first difference',
  'first prediction information',
  'second difference',
  'first prediction information',
  'second prediction information',
  'network parameter',
  'convolutional neural network',
  'first difference',
  'second difference',
  'method',
  'convolutional neural network',
  'method'],
 ['short-term power load prediction method',
  'power system',
  'actual power consumption',
  'measured power load',
  'predicted power consumption',
  'measured power load',
  'method',
  'deep belief network prediction model',
  'actual power consumption',
  'measured power load',
  'deep b

In [9]:
from gensim import corpora

dictionary = corpora.Dictionary(patent_content_tokenized)
dictionary.save('deeplearning.dict')
print(dictionary)

Dictionary(31 unique tokens: ['convolutional neural network', 'first difference', 'first prediction information', 'method', 'network parameter']...)


In [10]:
corpus = [dictionary.doc2bow(text) for text in patent_content_tokenized]
corpora.MmCorpus.serialize('deeplearning.mm', corpus)
pprint(corpus)

[[(0, 5),
  (1, 3),
  (2, 3),
  (3, 4),
  (4, 2),
  (5, 2),
  (6, 2),
  (7, 3),
  (8, 2),
  (9, 2)],
 [(3, 1),
  (10, 1),
  (11, 3),
  (12, 5),
  (13, 2),
  (14, 5),
  (15, 2),
  (16, 2),
  (17, 2),
  (18, 2),
  (19, 2),
  (20, 1)],
 [(3, 1),
  (10, 1),
  (18, 1),
  (20, 2),
  (21, 2),
  (22, 3),
  (23, 2),
  (24, 2),
  (25, 2),
  (26, 4),
  (27, 2),
  (28, 2),
  (29, 2),
  (30, 2)]]


In [11]:
%matplotlib inline
import matplotlib.pyplot as plt
from gensim import corpora, models, similarities, matutils
from wordcloud import WordCloud
NUM_TOPICS = 5
model = models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word = dictionary, alpha = None)

model.print_topics(5)

[(0,
  '0.032*"method" + 0.032*"deep belief network prediction model" + 0.032*"convolutional neural network" + 0.032*"prediction" + 0.032*"second difference" + 0.032*"actual power consumption" + 0.032*"measured power load" + 0.032*"predicted coordinate" + 0.032*"training" + 0.032*"neural network"'),
 (1,
  '0.152*"convolutional neural network" + 0.123*"method" + 0.094*"first prediction information" + 0.094*"first difference" + 0.094*"second difference" + 0.064*"second prediction information" + 0.064*"network parameter" + 0.064*"original image" + 0.064*"scrambled sample image" + 0.064*"target object"'),
 (2,
  '0.090*"predicted coordinate" + 0.069*"base station" + 0.057*"training" + 0.053*"deep belief network prediction model" + 0.048*"measured power load" + 0.047*"actual coordinate" + 0.047*"real value" + 0.047*"user" + 0.047*"small base station" + 0.047*"control method"'),
 (3,
  '0.144*"measured power load" + 0.135*"deep belief network prediction model" + 0.087*"actual power consumpt

In [82]:
print("length of corpus——",len(corpus))

#输出模型及模型中的各主题包含的词
topictermlist = model.print_topics(-1)
pprint(topictermlist)

length of corpus—— 3
[(0,
  '0.151*"convolutional neural network" + 0.121*"method" + 0.093*"second '
  'difference" + 0.093*"first prediction information" + 0.092*"first '
  'difference" + 0.065*"scrambled sample image" + 0.064*"network parameter" + '
  '0.064*"original image" + 0.064*"second prediction information" + '
  '0.063*"target object"'),
 (1,
  '0.123*"predicted coordinate" + 0.094*"base station" + 0.064*"training" + '
  '0.064*"small base station" + 0.064*"sample error" + 0.064*"actual '
  'coordinate" + 0.064*"real value" + 0.064*"neural network" + 0.064*"control '
  'method" + 0.064*"deep neural network"'),
 (2,
  '0.032*"method" + 0.032*"deep belief network prediction model" + '
  '0.032*"convolutional neural network" + 0.032*"measured power load" + '
  '0.032*"prediction" + 0.032*"predicted coordinate" + 0.032*"first prediction '
  'information" + 0.032*"first difference" + 0.032*"actual power consumption" '
  '+ 0.032*"output"'),
 (3,
  '0.065*"convolutional neural netw

In [12]:
for topic in model.print_topics(num_words=10):
    print(topic[1])

0.032*"method" + 0.032*"deep belief network prediction model" + 0.032*"convolutional neural network" + 0.032*"prediction" + 0.032*"second difference" + 0.032*"actual power consumption" + 0.032*"measured power load" + 0.032*"predicted coordinate" + 0.032*"training" + 0.032*"neural network"
0.152*"convolutional neural network" + 0.123*"method" + 0.094*"first prediction information" + 0.094*"first difference" + 0.094*"second difference" + 0.064*"second prediction information" + 0.064*"network parameter" + 0.064*"original image" + 0.064*"scrambled sample image" + 0.064*"target object"
0.090*"predicted coordinate" + 0.069*"base station" + 0.057*"training" + 0.053*"deep belief network prediction model" + 0.048*"measured power load" + 0.047*"actual coordinate" + 0.047*"real value" + 0.047*"user" + 0.047*"small base station" + 0.047*"control method"
0.144*"measured power load" + 0.135*"deep belief network prediction model" + 0.087*"actual power consumption" + 0.065*"input" + 0.065*"short-term 

In [72]:
#text = ["user"]
document_output=[]
for text in patent_content_tokenized:
    bow = dictionary.doc2bow(text)
    #print("get_document_topics", model.get_document_topics(bow))
    d_topic=model.get_document_topics(bow)
    print(d_topic)
    #NUM_TOPICS代表最大主题个数，如何将其余主题为0的值补充进来？，如第1个文档里，补充[0,0],[2,0][3,0][4,0]
    
    document_output.append(d_topic)
for index,pro in enumerate(document_output):
    print(index,pro)

[(1, 0.97239953)]
[(3, 0.9722375)]
[(2, 0.9723593)]
0 [(1, 0.97239953)]
1 [(3, 0.9722375)]
2 [(2, 0.9723593)]
